In [ ]:
!pip install seaborn
import seaborn as sns

import pandas as pd


# Load the CSV files into DataFrames
df_togo = pd.read_csv('/content/togo-dapaong_qc.csv')
df_sierraleone = pd.read_csv('/content/sierraleone-bumbuna.csv')
df_benin = pd.read_csv('/content/benin-malanville.csv')

# Display the first few rows of each DataFrame
print("Togo - Dapaong Data:")
display(df_togo.head())

print("\nSierra Leone - Bumbuna Data:")
display(df_sierraleone.head())

print("\nBenin - Malanville Data:")
display(df_benin.head())
# Check the structure of each DataFrame
print("Togo - Dapaong Info:")
df_togo.info()

print("\nSierra Leone - Bumbuna Info:")
df_sierraleone.info()

print("\nBenin - Malanville Info:")
df_benin.info()

# Get basic statistics for each DataFrame
print("Togo - Dapaong Statistics:")
display(df_togo.describe())

print("\nSierra Leone - Bumbuna Statistics:")
display(df_sierraleone.describe())

print("\nBenin - Malanville Statistics:")
display(df_benin.describe())
# Check for missing values in each DataFrame
print("Missing values in Togo - Dapaong:")
print(df_togo.isnull().sum())

print("\nMissing values in Sierra Leone - Bumbuna:")
print(df_sierraleone.isnull().sum())

print("\nMissing values in Benin - Malanville:")
print(df_benin.isnull().sum())

# Example: Fill missing values with forward fill
df_togo.fillna(method='ffill', inplace=True)
df_sierraleone.fillna(method='ffill', inplace=True)
df_benin.fillna(method='ffill', inplace=True)
# Convert Timestamp to datetime format and set as index
df_togo['Timestamp'] = pd.to_datetime(df_togo['Timestamp'])
df_togo.set_index('Timestamp', inplace=True)

df_sierraleone['Timestamp'] = pd.to_datetime(df_sierraleone['Timestamp'])
df_sierraleone.set_index('Timestamp', inplace=True)

df_benin['Timestamp'] = pd.to_datetime(df_benin['Timestamp'])
df_benin.set_index('Timestamp', inplace=True)
# Display the first few rows to verify changes
print("Togo - Dapaong Data (after preprocessing):")
display(df_togo.head())

print("\nSierra Leone - Bumbuna Data (after preprocessing):")
display(df_sierraleone.head())

print("\nBenin - Malanville Data (after preprocessing):")
display(df_benin.head())
import matplotlib.pyplot as plt

# Plot GHI, DNI, and DHI over time for each region
def plot_irradiance(df, region_name):
    plt.figure(figsize=(14, 6))
    plt.plot(df.index, df['GHI'], label='GHI (W/m²)', color='orange')
    plt.plot(df.index, df['DNI'], label='DNI (W/m²)', color='red')
    plt.plot(df.index, df['DHI'], label='DHI (W/m²)', color='blue')
    plt.title(f'Solar Irradiance Over Time in {region_name}')
    plt.xlabel('Date')
    plt.ylabel('Irradiance (W/m²)')
    plt.legend()
    plt.show()

# Plot for each region
plot_irradiance(df_togo, 'Togo - Dapaong')
plot_irradiance(df_sierraleone, 'Sierra Leone - Bumbuna')
plot_irradiance(df_benin, 'Benin - Malanville')
# Plot Temperature and Humidity over time for each region
def plot_temp_humidity(df, region_name):
    plt.figure(figsize=(14, 6))
    plt.plot(df.index, df['Tamb'], label='Ambient Temperature (°C)', color='red')
    plt.plot(df.index, df['RH'], label='Relative Humidity (%)', color='blue')
    plt.title(f'Temperature and Humidity Over Time in {region_name}')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C) / Humidity (%)')
    plt.legend()
    plt.show()

# Plot for each region
plot_temp_humidity(df_togo, 'Togo - Dapaong')
plot_temp_humidity(df_sierraleone, 'Sierra Leone - Bumbuna')
plot_temp_humidity(df_benin, 'Benin - Malanville')

# Plot Wind Speed and Direction over time for each region
def plot_wind_conditions(df, region_name):
    plt.figure(figsize=(14, 6))
    plt.plot(df.index, df['WS'], label='Wind Speed (m/s)', color='green')
    plt.plot(df.index, df['WD'], label='Wind Direction (°N)', color='purple')
    plt.title(f'Wind Conditions Over Time in {region_name}')
    plt.xlabel('Date')
    plt.ylabel('Wind Speed (m/s) / Wind Direction (°N)')
    plt.legend()
    plt.show()

# Plot for each region
plot_wind_conditions(df_togo, 'Togo - Dapaong')
plot_wind_conditions(df_sierraleone, 'Sierra Leone - Bumbuna')
plot_wind_conditions(df_benin, 'Benin - Malanville')



# Correlation matrix for each region
def plot_correlation_matrix(df, region_name):
    corr_matrix = df.corr()
    plt.figure(figsize=(10, 8))
    plt.title(f'Correlation Matrix for {region_name}')
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f') # Now sns is defined and can be used
    plt.show()


# Plot for each region
plot_correlation_matrix(df_togo, 'Togo - Dapaong')
plot_correlation_matrix(df_sierraleone, 'Sierra Leone - Bumbuna')
plot_correlation_matrix(df_benin, 'Benin - Malanville')


# Plot for each region
plot_correlation_matrix(df_togo, 'Togo - Dapaong')
plot_correlation_matrix(df_sierraleone, 'Sierra Leone - Bumbuna')
plot_correlation_matrix(df_benin, 'Benin - Malanville')


Output hidden; open in https://colab.research.google.com to view.

In [19]:
# Install the required package
!pip install python-docx

from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_BREAK

# Create a new Document
doc = Document()

# Add Title
doc.add_heading('Strategic Recommendation Report', 0).alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Add Executive Summary
doc.add_heading('Executive Summary', level=1)
doc.add_paragraph(
    "MoonLight Energy Solutions aims to enhance operational efficiency and sustainability through targeted solar "
    "investments. Our analysis of environmental data from three regions—Togo (Dapaong), Sierra Leone (Bumbuna), and "
    "Benin (Malanville)—has identified key trends that will guide the company's strategic decisions. This report provides "
    "recommendations for the optimal region(s) for solar installation, taking into account solar irradiance, temperature, "
    "humidity, and wind conditions."
)

# Add Data Overview and Key Findings
doc.add_heading('1. Overview of Analysis', level=1)
doc.add_heading('1.1 Data Overview', level=2)
doc.add_paragraph(
    "The dataset analyzed includes solar radiation measurements (GHI, DNI, DHI), temperature, humidity, wind speed, and "
    "direction, along with additional environmental factors like barometric pressure and precipitation. The primary goal was "
    "to identify the region with the most favorable conditions for solar panel installation."
)

doc.add_heading('1.2 Key Findings', level=2)
doc.add_paragraph(
    "• Togo (Dapaong): Exhibits the highest and most consistent solar irradiance, with GHI peaking consistently. The "
    "temperature is moderate, and humidity levels are within an optimal range.\n"
    "• Sierra Leone (Bumbuna): Shows moderate solar irradiance with higher variability. Temperature and humidity are "
    "slightly higher, which could impact solar panel efficiency.\n"
    "• Benin (Malanville): Displays lower solar irradiance compared to Togo but has relatively stable conditions. Wind "
    "speed and direction are favorable, but temperature extremes might affect performance."
)

# Add Detailed Analysis
doc.add_heading('2. Detailed Analysis', level=1)

doc.add_heading('2.1 Solar Irradiance', level=2)
doc.add_paragraph(
    "Togo (Dapaong) stands out with the highest GHI values, indicating strong potential for solar energy generation. "
    "The consistency in DNI also suggests that this region receives direct sunlight without significant obstruction.\n"
    "Recommendation: Prioritize Togo for solar panel installation due to its superior solar irradiance levels."
)

doc.add_heading('2.2 Temperature and Humidity', level=2)
doc.add_paragraph(
    "Togo (Dapaong) and Benin (Malanville) have moderate temperature levels, which are ideal for solar panel efficiency. "
    "However, Sierra Leone (Bumbuna) shows higher humidity levels, which might affect the durability of solar panels over time.\n"
    "Recommendation: Favor regions with moderate temperatures like Togo and Benin to optimize panel performance and longevity."
)

doc.add_heading('2.3 Wind Conditions', level=2)
doc.add_paragraph(
    "Benin (Malanville) has favorable wind conditions with moderate wind speeds and consistent wind direction, which "
    "can help in cooling solar panels and reducing thermal stress.\n"
    "Recommendation: Consider Benin for installations that might benefit from wind cooling, though additional structural "
    "support may be necessary for regions with higher wind variability."
)

doc.add_heading('2.4 Correlation Analysis', level=2)
doc.add_paragraph(
    "Strong positive correlations between solar irradiance (GHI, DNI) and temperature were observed in all regions. "
    "Wind speed showed a negative correlation with irradiance, indicating that higher wind speeds might reduce the amount "
    "of solar energy received.\n"
    "Recommendation: Focus on regions where the correlation between temperature and irradiance is favorable, with minimal "
    "negative impact from wind conditions."
)

# Add Strategic Recommendations
doc.add_heading('3. Strategic Recommendations', level=1)

doc.add_heading('3.1 Region Selection', level=2)
doc.add_paragraph(
    "Primary Recommendation: Togo (Dapaong) emerges as the most favorable region for solar installations due to its high "
    "and consistent solar irradiance, moderate temperatures, and optimal humidity levels.\n"
    "Secondary Recommendation: Benin (Malanville) is a viable option for installations where wind cooling might be beneficial, "
    "though temperature extremes should be managed."
)

doc.add_heading('3.2 Operational Efficiency', level=2)
doc.add_paragraph(
    "Panel Placement: In Togo, panels should be placed in areas that maximize exposure to direct sunlight, taking into account "
    "the direction and intensity of solar radiation throughout the day.\n"
    "Technology Choice: Use solar panels that are optimized for high-temperature environments to mitigate any potential efficiency loss."
)

doc.add_heading('3.3 Risk Mitigation', level=2)
doc.add_paragraph(
    "Wind Protection: In regions like Benin with variable wind conditions, consider using reinforced structures or tracking systems "
    "that can adjust to changing wind directions.\n"
    "Maintenance Strategy: Implement a regular maintenance schedule in Sierra Leone to manage the effects of higher humidity on solar panel performance."
)

# Add Conclusion
doc.add_heading('4. Conclusion', level=1)
doc.add_paragraph(
    "The analysis clearly identifies Togo (Dapaong) as the prime candidate for solar investments, aligning with MoonLight "
    "Energy Solutions' goal of enhancing operational efficiency and sustainability. By prioritizing this region and considering "
    "the secondary option of Benin, the company can achieve significant gains in solar energy generation while managing risks effectively."
)

# Save the document
doc.save('Strategic_Recommendation_Report.docx')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.1 MB/s eta 0:00:00
